In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
from scipy.stats import poisson,skellam



In [2]:
from sklearn.model_selection import train_test_split

gamedata = pd.read_csv('/Users/mgor/Downloads/game.csv', error_bad_lines=False)
gamedata = gamedata[['home_team_id','away_team_id','home_goals','away_goals']]
train, test = train_test_split(gamedata, test_size=0.1)
gamedata = train
# print(type(train))
# print(train)
# print(test)
# print(next(test.iterrows())[1])
gamedata = gamedata.rename(columns={'home_goals': 'HomeGoals', 'away_goals': 'AwayGoals'})
gamedata.head()

,home_team_id,away_team_id,HomeGoals,AwayGoals
6652,10,52,2,4
2120,17,2,1,3
2973,18,25,2,5
1819,25,16,2,3
3213,1,15,2,1


In [3]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

goal_model_data = pd.concat([gamedata[['home_team_id','away_team_id','HomeGoals']].assign(home=1).rename(
            columns={'home_team_id':'team', 'away_team_id':'opponent','HomeGoals':'goals'}),
           gamedata[['away_team_id','home_team_id','AwayGoals']].assign(home=0).rename(
            columns={'away_team_id':'team', 'home_team_id':'opponent','AwayGoals':'goals'})])

poisson_model = smf.glm(formula="goals ~ home + team + opponent", data=goal_model_data, 
                        family=sm.families.Poisson()).fit()
poisson_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  goals   No. Observations:                13392
Model:                            GLM   Df Residuals:                    13388
Model Family:                 Poisson   Df Model:                            3
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -25025.
Date:                Fri, 16 Nov 2018   Deviance:                       14292.
Time:                        22:05:40   Pearson chi2:                 1.25e+04
No. Iterations:                     4   Covariance Type:             nonrobust
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.9618      0.012     77.252      0.000       0.937       0.986
home           0.0995      0.010      9.556      0.000       0.079       0.120
team          -0.0002      0.000     -0.362      0.717      -0.001       0.001
opponent       0.0004      0.000      0.874      0.382      -0.000       0.001
==============================================================================
"""

In [4]:
from sklearn.metrics import mean_squared_error

actuals_home = []
predictions_home = []
actuals_away = []
predictions_away = []
for row in test.iterrows():
#     print(row[1].home_team_id)
#     print(poisson_model.predict(pd.DataFrame(data={'team': row[1].home_team_id, 'opponent': row[1].away_team_id,
#                                        'home':1},index=[1])).values)
    actuals_home.append(row[1].home_goals)
    actuals_away.append(row[1].away_goals)
    predictions_home.append(poisson_model.predict(pd.DataFrame(data={'team': row[1].home_team_id, 'opponent': row[1].away_team_id,
                                       'home':1},index=[1])).values[0])
    predictions_away.append(poisson_model.predict(pd.DataFrame(data={'team': row[1].away_team_id, 'opponent': row[1].home_team_id,
                                       'home':0},index=[1])).values[0])
    
    

print(mean_squared_error(actuals_home, predictions_home))
print(mean_squared_error(actuals_away, predictions_away))

2.965892159966072
2.655573375782476
